<a href="https://colab.research.google.com/github/Vishy-A/MachineLearningIntro/blob/main/HW6/4105HW6P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [482]:
#Vishy Adusumilli
#801185663
#HW Problem 1

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict
from google.colab import drive


drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [483]:
varsall = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus', 'price']
varstrings = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

def binarymap(inp):
  return inp.map({'yes' : 1, 'no' : 0, 'furnished' : 2, 'semi-furnished' : 1, 'unfurnished' : 0 })

dataset[varstrings] = dataset[varstrings].apply(binarymap)


dataset.head()


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [484]:
standardScaler = StandardScaler()
minMaxScaler = MinMaxScaler()

dataset[varsall] = minMaxScaler.fit_transform(dataset[varsall])
np.random.seed(0)
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.000000,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,1.0
1,0.909091,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,1.0
2,0.909091,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,0.5
3,0.906061,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,1.0
4,0.836364,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,1.0


In [485]:
y = dataset.pop('price')
x = dataset.values
dataset.head()


,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,1.0
1,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,1.0
2,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,0.5
3,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,1.0
4,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,1.0


In [486]:
x = torch.tensor(x)
y = torch.tensor(y)

In [487]:
samples = x.shape[0]
n_val = int(.2 * samples)

shuffledindices = torch.randperm(samples)

trainindices = shuffledindices[:-n_val]
valindices = shuffledindices[-n_val:]

trainindices, valindices

(tensor([308, 420, 285, 221,  49, 242, 223, 117, 328, 406, 249, 399, 330, 353,
         362, 523, 483, 299, 296, 194, 466, 400, 206,  25,  43, 163, 428, 527,
         322, 218, 178,  36, 391, 532, 198, 161, 478, 540,  47, 448, 333, 279,
         208, 411,  23, 311, 374,  42, 366, 390, 227, 111, 397, 316, 271, 513,
          75, 364, 528, 463, 349, 539, 535, 522, 162, 149, 304,  15, 132, 103,
         455, 205, 515, 225, 203, 541, 408,  45, 137, 287, 321, 405, 394, 491,
         126, 160,  22, 325,  57, 286,  89, 457, 444, 467, 380, 510, 235, 290,
         150, 130, 291, 337, 247, 168, 395, 239, 342, 509, 237, 156, 344, 252,
          95, 338, 276,  11, 459, 413, 143,  40,  37, 294, 497, 529, 155, 139,
         458, 165,  63,  55,  67,  71, 306, 536, 418, 288, 255, 186, 248, 309,
         112, 442, 179,  90, 456, 351, 473, 250, 447, 295, 450, 460, 496,  88,
           0, 431, 385,  17, 173, 264,  60, 542, 498, 314, 215, 193, 367, 230,
          94, 512, 386, 119, 507, 341, 275,  14, 373

In [488]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]


In [489]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]

xntrain = .1 * xtrain
xnval = .1 * valx

In [490]:
def training_loop(epochs, optimizer, model, loss_func, xtrain, ytrain, xtest, ytest):
  for epoch in range(1, epochs + 1):
    t_p_train = model(xtrain)
    loss_train = loss_func(t_p_train, ytrain)

    t_p_val = model(xtest)
    vloss = loss_func(t_p_val, ytest)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()


    if epoch % 500 == 0 :
      print(f"Epoch {epoch}, Training Loss {loss_train.item():.4f}, " f" Validation Loss {vloss.item():.4f}")



In [491]:
def loss_func(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [492]:
sequential_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12,32)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(32, 1))
]))
sequential_model

Sequential(
  (hidden_linear): Linear(in_features=12, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=32, out_features=1, bias=True)
)

In [493]:
optimizer = optim.SGD(sequential_model.parameters(), lr = 1e-3)

training_loop(
    epochs = 5000,
    optimizer = optimizer,
    model = sequential_model,
    loss_func = nn.MSELoss(),
    xtrain = xntrain.to(torch.float32),
    ytrain = ytrain.to(torch.float32),
    xtest = xnval.to(torch.float32),
    ytest = valy.to(torch.float32)
)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training Loss 0.0262,  Validation Loss 0.0266
Epoch 1000, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 1500, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 2000, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 2500, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 3000, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 3500, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 4000, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 4500, Training Loss 0.0262,  Validation Loss 0.0265
Epoch 5000, Training Loss 0.0262,  Validation Loss 0.0265


In [494]:
sequential_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12,32)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear_2', nn.Linear(32, 64)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear_3', nn.Linear(64, 16)),
    ('outputlinear', nn.Linear(16,1))
]))
sequential_model

Sequential(
  (hidden_linear): Linear(in_features=12, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (hidden_linear_2): Linear(in_features=32, out_features=64, bias=True)
  (hidden_linear_3): Linear(in_features=64, out_features=16, bias=True)
  (outputlinear): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
optimizer = optim.SGD(sequential_model.parameters(), lr = 1e-3)

training_loop(
    epochs = 5000,
    optimizer = optimizer,
    model = sequential_model,
    loss_func = nn.MSELoss(),
    xtrain = xntrain.to(torch.float32),
    ytrain = ytrain.to(torch.float32),
    xtest = xnval.to(torch.float32),
    ytest = valy.to(torch.float32)
)

Epoch 500, Training Loss 0.0268,  Validation Loss 0.0281
Epoch 1000, Training Loss 0.0261,  Validation Loss 0.0266
Epoch 1500, Training Loss 0.0261,  Validation Loss 0.0265
Epoch 2000, Training Loss 0.0261,  Validation Loss 0.0265
Epoch 2500, Training Loss 0.0261,  Validation Loss 0.0265
Epoch 3000, Training Loss 0.0261,  Validation Loss 0.0265
Epoch 3500, Training Loss 0.0261,  Validation Loss 0.0265
